# Extracción de datos
En este notebook se tratará la extracción de datos de sitios web como Reddit o Github, mediante APIs expresamente hechas por el sitio, o funciones de Python que servirán para realizar las peticiones.

Para empezar, se cargarán los ajustes definidos en el apartado anterior junto con la importación de algunas librerías básicas necesarias para comenzar. 

In [1]:
import sys, os

#Carga del archivo setup.py
%run -i setup.py

#Carga del archivo settings.py
#%run "$BASE_DIR/settings.py"
%reload_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'png'

You are working on a local system.
Files will be searched relative to "..".


## Extracción de datos con la librería *request*

Para este primer ejemplo se va a hacer uso de la librería *request* que se incluye con la instalación de python. Esta librería es el método más básico para acceder y extraer información a través de una API, pero al mismo tiempo es una herramienta muy potente.

Véamos a continuación como se realiza una petición GET en la que se tratará de listar todos los repositorios que cuenten con unas características  determinadas. En este caso, por ejemplo, vamos a listar todos los repositorios relacionados con el protocolo Zigbee que, al no usar el token de autenticación de un usuario registrado, nos devolverá aquellos repositorios que sean públicos, si se quieren listar también aquellos privados, hay que proporcionar el token de acceso.

In [11]:
import requests
import json

response = requests.get('https://api.github.com/search/repositories',
    params={'q': 'zigbee'},
    headers={'Accept': 'application/vnd.github.v3.text-match+json'})

print(response.status_code)
print(response.json())

200
{'total_count': 3789, 'incomplete_results': False, 'items': [{'id': 170371837, 'node_id': 'MDEwOlJlcG9zaXRvcnkxNzAzNzE4Mzc=', 'name': 'zigbee2mqtt.io', 'full_name': 'Koenkk/zigbee2mqtt.io', 'private': False, 'owner': {'login': 'Koenkk', 'id': 2892853, 'node_id': 'MDQ6VXNlcjI4OTI4NTM=', 'avatar_url': 'https://avatars.githubusercontent.com/u/2892853?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/Koenkk', 'html_url': 'https://github.com/Koenkk', 'followers_url': 'https://api.github.com/users/Koenkk/followers', 'following_url': 'https://api.github.com/users/Koenkk/following{/other_user}', 'gists_url': 'https://api.github.com/users/Koenkk/gists{/gist_id}', 'starred_url': 'https://api.github.com/users/Koenkk/starred{/owner}{/repo}', 'subscriptions_url': 'https://api.github.com/users/Koenkk/subscriptions', 'organizations_url': 'https://api.github.com/users/Koenkk/orgs', 'repos_url': 'https://api.github.com/users/Koenkk/repos', 'events_url': 'https://api.github.com/users/Koe

Para poder entender de forma más clara la lista de repositorios obtenida, vamos a dar formato Markdown al archivo json y se mostrarán los 5 primeros resultados de la respuesta obtenida:

In [13]:
from IPython.display import Markdown, display

def printmd(string):
    display(Markdown(string))

for item in response.json()['items'][:5]:
    printmd('**' + item['name'] + '**' + ': repository ' +
            item['text_matches'][0]['property'] + ' - \"*' +
            item['text_matches'][0]['fragment'] + '*\" matched with ' + '**' +
            item['text_matches'][0]['matches'][0]['text'] + '**')

**zigbee2mqtt.io**: repository name - "*zigbee2mqtt.io*" matched with **zigbee2mqtt**

**zigbee2mqtt**: repository description - "*Zigbee 🐝 to MQTT bridge 🌉, get rid of your proprietary Zigbee bridges 🔨*" matched with **Zigbee**

**zigbee**: repository name - "*zigbee*" matched with **zigbee**

**zigbee-herdsman-converters**: repository description - "*Collection of device converters to be used with zigbee-herdsman*" matched with **zigbee**

**hassio-zigbee2mqtt**: repository name - "*hassio-zigbee2mqtt*" matched with **zigbee2mqtt**

### Listar comentarios del apartado *Issues* de un repositorio
Es posible listar todos los comentarios del apartado Issues de un repositorio si así se especifica en la petición el nombre del repositorio y el dueño del mismo:

(He buscado el repositorio en la web de github porque el nombre del repo no era igual)

In [33]:
response = requests.get(
    'https://api.github.com/repos/zigbee2mqtt/hassio-zigbee2mqtt/issues/comments')
print('Response Code', response.status_code)
print('Number of comments', len(response.json()))

Response Code 403
Number of comments 2


Se observa que únicamente ha recopilado 30 comentarios, esto es porque la API de github limita el número de elementos para cada respuesta. Si mostramos por pantalla los enlaces de la respuesta obtendremos el número de páginas contenidas en la respuesta.

In [34]:
response.links

{}

### Paginación
Se usa la paginación para limitar el número de elementos en una que se devolverán tras una petición.

Al mostrar los links de *response* se ve que proporciona el enlace a la siguiente página a la respuesta y a la última del total.

Para obtener todos los resultados se debe definir una función que llame a la siguiente página, y así hasta que se hayan procesado todas de forma recursiva.

Del mismo modo, importaremos *Pandas* a nuestro código para transformar los datos obtenidos en un Data Frame.

In [35]:
import pandas as pd

def get_all_pages(url, params=None, headers=None):
    output_json = []
    response = requests.get(url, params=params, headers=headers)
    if response.status_code == 200:
        output_json = response.json()
        if 'next' in response.links:
            next_url = response.links['next']['url']
            if next_url is not None:
                output_json += get_all_pages(next_url, params, headers)
    return output_json


out = get_all_pages(
    "https://api.github.com/repos/zigbee2mqtt/hassio-zigbee2mqtt/issues/comments",
    params={
        #Como parámetros indicamos la fecha desde la cuál queremos obtener los resultados,
        #el orden (de creación) y la dirección
        'since': '2020-07-01T10:00:01Z',
        'sorted': 'created',
        'direction': 'desc'
    },
    headers={'Accept': 'application/vnd.github.v3+json'})

#Los resultados obtenidos los transformamos en un Data Frame para su posterior análisis
df = pd.DataFrame(out)
print(out[:5])
pd.set_option('display.max_colwidth', 1)

# #Muestra por pantalla el total de resultados
print(df['body'].count())

# #Muestra por pantalla algún ejemplo de los resultados obtenidos
df[['id', 'created_at', 'body']].sample(1, random_state=42)

[]


KeyError: 'body'